In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10
SEED = 123


In [23]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print("Znalezione klasy:", class_names)


Found 20638 files belonging to 15 classes.
Using 16511 files for training.
Found 20638 files belonging to 15 classes.
Using 4127 files for validation.
Znalezione klasy: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [24]:
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [25]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True

# (opcjonalnie zamroź pierwsze 50 warstw)
for layer in base_model.layers[:50]:
    layer.trainable = False

model = models.Sequential([
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])


In [26]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [27]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop]
)


Epoch 1/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 493s 200ms/step - accuracy: 0.1817 - loss: 2.5358 - val_accuracy: 0.3165 - val_loss: 2.2341
Epoch 2/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 322s 156ms/step - accuracy: 0.4566 - loss: 1.8170 - val_accuracy: 0.4417 - val_loss: 1.7314
Epoch 3/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 332s 161ms/step - accuracy: 0.6019 - loss: 1.3314 - val_accuracy: 0.5619 - val_loss: 1.3397
Epoch 4/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 336s 163ms/step - accuracy: 0.6815 - loss: 1.0411 - val_accuracy: 0.6416 - val_loss: 1.0836
Epoch 5/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 369s 179ms/step - accuracy: 0.7312 - loss: 0.8729 - val_accuracy: 0.6802 - val_loss: 0.9982
Epoch 6/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 611s 296ms/step - accuracy: 0.7701 - loss: 0.7418 - val_accuracy: 0.6712 - val_loss: 1.0029
Epoch 7/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 503s 244ms/step - accuracy: 0.7907 - loss: 0.6764 - val_accuracy: 0.7381 - val_loss: 0.8083
Epoch 8/10
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 343s 166ms/step - ac

In [28]:
loss, acc = model.evaluate(val_ds)
print(f"Dokładność walidacji: {acc * 100:.2f}%")

516/516 ━━━━━━━━━━━━━━━━━━━━ 34s 66ms/step - accuracy: 0.7485 - loss: 0.7788
Dokładność walidacji: 75.19%
